# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [12]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4.1-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'partner page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [13]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-4.1-nano
Found 5 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'career page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'blog or updates', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'professional profile',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [14]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4.1-nano
Found 15 relevant links


{'links': [{'url': 'https://huggingface.co/', 'type': 'homepage'},
  {'url': 'https://huggingface.co/models', 'type': 'models'},
  {'url': 'https://huggingface.co/datasets', 'type': 'datasets'},
  {'url': 'https://huggingface.co/spaces', 'type': 'spaces'},
  {'url': 'https://huggingface.co/docs', 'type': 'documentation'},
  {'url': 'https://huggingface.co/enterprise', 'type': 'enterprise'},
  {'url': 'https://huggingface.co/pricing', 'type': 'pricing'},
  {'url': 'https://huggingface.co/inference/models', 'type': 'models info'},
  {'url': 'https://endpoints.huggingface.co', 'type': 'API endpoints'},
  {'url': 'https://huggingface.co/huggingface', 'type': 'company profile'},
  {'url': 'https://discuss.huggingface.co', 'type': 'community forum'},
  {'url': 'https://status.huggingface.co/', 'type': 'status page'},
  {'url': 'https://github.com/huggingface', 'type': 'GitHub repository'},
  {'url': 'https://twitter.com/huggingface', 'type': 'Twitter'},
  {'url': 'https://www.linkedin.com/co

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [15]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [16]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-4.1-nano
Found 15 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Tongyi-MAI/Z-Image-Turbo
Updated
1 day ago
•
217k
•
2.45k
microsoft/VibeVoice-Realtime-0.5B
Updated
1 day ago
•
57k
•
608
deepseek-ai/DeepSeek-V3.2
Updated
9 days ago
•
33.1k
•
847
zai-org/GLM-4.6V-Flash
Updated
about 14 hours ago
•
6.92k
•
260
alibaba-pai/Z-Image-Turbo-Fun-Controlnet-Union
Updated
8 days ago
•
289
Browse 1M+ models
Spaces
Running
on
Zero
MCP
Featured
1.28k
Z Image Turbo
🏃
1.28k
Generate images from text prompts
Running
on
Zero
449
Z Image Turbo
🖼
449
Generate stunning images from text prompts
Running
169
Evaluatio

In [ ]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4.1-nano
Found 1 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTongyi-MAI/Z-Image-Turbo\nUpdated\n1 day ago\n•\n217k\n•\n2.45k\nmicrosoft/VibeVoice-Realtime-0.5B\nUpdated\n1 day ago\n•\n57k\n•\n608\ndeepseek-ai/DeepSeek-V3.2\nUpdated\n9 days ago\n•\n33.1k\n•\n848\nzai-org/GLM-4.6V-Flash\nUpdated\nabout 14 hours ago\n•\n6.92k\n•\n261\nalibaba-pai/Z-Image-Turbo-Fun-Controlnet-Union\nUpdated\n8 days ago\n•\n289\nBrowse 1M+ models\nSpaces\nRunning\non\nZer

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4.1-nano
Found 15 relevant links


# Hugging Face: The AI Playground Where Machines and People Hug It Out 🤗

Welcome to **Hugging Face**, where AI isn’t just smart—it’s community smart! Imagine a place where machine learning models, datasets, and apps have a never-ending party, and every data whisperer, AI wizard, and tech dreamer is invited to join the fun.

---

## What’s Hugging Face All About?

**Hugging Face** is *the* hub for the global AI community to build the future—together. Whether you’re a researcher, developer, or just AI-curious, here’s the deal:

- **Models Galore:** More than **1 million** ready-to-rock AI models. Need text generation? Done. Fancy turning text into stunning images or videos? Easy. From NLP wizards like Transformers to the latest in image and audio, it's all here.

- **Datasets Galore:** Over **250,000 datasets** spanning every imaginable AI need—training your next model just got way easier.

- **Spaces:** Your very own AI app corners—run cool demos, share interactive apps, and showcase your AI masterpieces, all hosted by Hugging Face’s powerful infrastructure.

- **Community Spirit:** Not just a platform, but a thriving hive where AI folks collaborate, share ideas, and push boundaries.

---

## Meet the Team: AI Avengers with a Smile

Hugging Face isn’t some distant, robot-run factory. It’s a vibrant, welcoming crew led by passionate folks who love open-source and disruptive innovation. With security, privacy, and enterprise-grade performance baked in, the team caters to hobbyists and Fortune 500ers alike.

- Collaboration is king 👑 —from users contributing models and datasets to integration with major players like Microsoft, Nvidia, Alibaba, and OpenAI.

- Enterprise-grade features too — think Single Sign-On, audit logs, private dataset viewers, and cloud-scale compute power—all ready to scale your AI ambitions.

---

## Why Join Hugging Face? (Besides the Free Virtual Hugs)

- **Build your AI portfolio:** Share your models, apps, and datasets to get noticed by peers, employers, or that mysterious tech recruiter searching for talent.

- **Boost your projects:** Tap into free and paid compute resources (hello, ZeroGPU boosts!) so you don’t have to beg your laptop for mercy.

- **Be part of the AI revolution:** Collaborate with hundreds of thousands of users and tens of thousands of organizations pushing AI forward.

- **Career opportunities:** Whether you’re a machine learning engineer, developer relations hero, or a community whisperer, Hugging Face is growing fast—and looking for bright minds to join the journey.

---

## What Customers Say: They Love Us, We Hug Back

- **“Gamechanger for our AI pipeline!”** — Enterprise clients praise the platform’s seamless integration and enterprise security.

- **“Finally, a community where I’m not just ‘another user’.”** — Devs revel in the collaborative spirit and the treasure trove of open-source goodness.

---

## Pricing: Hug Your Budget Tight 🤗

- Individual explorers? A generous free tier with access to thousands of models and datasets.

- Teams start at just **$20/user/month** for advanced tools, security, and premium support.

- Enterprises, contact sales for custom contracts that make scaling AI as easy as hugging your favorite plushie.

---

# Join Us and Hug the Future

Whether you want to:

- **Build jaw-dropping AI models**

- **Collaborate with AI nerds around the globe**

- **Show off your AI apps and datasets**

Hugging Face is the place that gets AI people.

**Sign up today** and get ready to 🤗 your way into the future!

---

Website: [huggingface.co](https://huggingface.co)  
Community: Where AI loves to hang out.  
Careers: Bring your brain and a sense of humor. Help us hug the world smarter.

---

*Because building the future’s better with friends. And a few smiles.*

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4.1-nano
Found 16 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Welcome to Hugging Face! 🤗

**The AI Community Building the Future...and Probably Your Favorite Meme!**

---

## Who Are We?

Imagine a bustling hub of innovation where artificial intelligence and machine learning enthusiasts come together, argue about cat videos, and maybe, just maybe, build the next big thing. That's Hugging Face for you! We’re not just a platform; we’re *the* entertaining cornucopia of models, datasets, and gallant AI applications – all curated with a pinch of humor and a sprinkle of endless collaboration.

## What Can You Expect?

### 🚀 A Platform Like No Other
- **1 Million+ Models**: From image generation to voice synthesis, we've got more models than your favorite 90s sitcom had plot twists.
- **Datasets Galore**: Explore over 250k datasets that can help you train your AI...or just waste your time looking at interesting data patterns.
- **Spaces**: Showcase your genius in AI Apps. Who knows, maybe your creation will go viral and you'll be the talk of the AI world!

### 🤝 Community Vibes
Join a community that embraces shared knowledge like it's a group hug. Collaborate, contribute, and occasionally argue (but for research purposes, of course) with fellow AI aficionados. 

### 🎉 Work Culture 
Picture this: a bright, open environment where creativity flourishes faster than your sore back can complain. At Hugging Face, we love our team members and their quirky personal choices, whether that’s obsessively collecting rubber ducks or wielding an overuse of coffee cups as accessories.

## Careers at Hugging Face
Thinking of joining our ever-growing family? If you love AI and can't resist a clever pun, we want you! Positions range from AI researchers to community managers and include nobody asking you to produce a chart or graph just for fun. Explore your potential with us, and remember, the eggs you scramble in the morning could be the next breakthrough in AI!

## Who Are Our Customers?
We cater to the curious! From researchers in labs frantically seeking the next discovery to tech startups hoping to build a chatbot that remembers your birthday, our customers come from all walks of life. We even have that one hipster who uses AI to recreate vintage 80’s cartoons – so, there’s something for everyone!

## Join Us!
So, if you're looking to tickle your fancy (and your funny bone) while contributing to the exciting world of AI, come sign up with Hugging Face! Don't worry—no actual hugging required!

---

#### Ready to dive into AI magic? Visit us at [Hugging Face](https://huggingface.co) and let’s build the future together!

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-4.1-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Crafting Tomorrow! 🌟

## Who We Are
At Hugging Face, we’re more than just a company; we're a vibrant community of AI enthusiasts and machine learning magicians ready to change the world! We host something that resembles a modern-day Hogwarts for ML wizards, where collaboration is key, and creativity runs wild. 

## What We Do
Ever wanted to create an AI model that serenades you while you work? Or perhaps generate stunning images from the depths of your imagination? At Hugging Face, **we’ve got over 1 million models** just waiting for your next genius idea (or the cat meme you always wanted to create!). 

### Key Features:
- **Collaboration**: Join forces with fellow AI wizards to create models and datasets. Because why solve problems alone when you can struggle with friends?
- **Unlimited Creativity**: Text, images, videos, audio – if you can dream it, you can probably create it on our platform! 
- **Enterprise Solutions**: For organizations with grand aspirations and slightly larger budgets, we provide a variety of enterprise features designed to make you feel like the superhero of data! 🦸‍♂️

## The Hugging Community 💖
We believe in sharing cuddles, knowledge, and best practices. Our community is your go-to space for networking, learning, and occasionally arguing over which AI model is the best (we all know it’s yours).

## Careers at Hugging Face
Looking to join the magical realm of machine learning? We’re hiring! If you love algorithms as much as cat videos and can handle a bit of quirkiness, **Hugging Face could be your next home**. Our team values diversity, collaboration, and the occasional meme break.

### Why Work With Us?
- **Flexible Working Hours**: Perfect for night owls and early birds!
- **Supportive Culture**: We believe in lifting each other up – kind of like a group hug, but with code.
- **Cutting-Edge Projects**: Work on exciting technology that can change industries (and make friends along the way!).

## Join Us!
Whether you're a budding ML enthusiast, a seasoned developer, or just someone who loves to play with AI, **Hugging Face is where you need to be**! Visit our website to explore more, collaborate, or find the perfect role that makes you say, "This is my jam!" 

### In Conclusion
At Hugging Face, we don’t just write code; we create dreams, models, and a brighter future in AI. So, are you ready to join the hug? 🤗

**Explore more at [Hugging Face](https://huggingface.co)** or join our community today!